In [52]:
import pandas as pd
import numpy as np
from itertools import compress

In [53]:
def tsv_to_dataframe(file_tsv):
    '''
    this function transforms Eurostat tsv file in pandas dataframe
    file_tsv: file name. It's work with tsv and compressed file "tsv.gz"
    '''
    
    def clean_cells(x):
        '''This function transforms Eurostat Missing Values ": " in numpy missing values.
        Then clean Eurostat annotation "b, u, .."'''
        try:
            return float(x)
        except:
            try:
                return float(x.split(" ")[0])
            except:
                return np.nan
    # open the Eurostat TSV file 

    data = pd.read_csv(file_tsv, sep="\t")
    # Create a dataframe for values data
    data_clean = data
    # Clean data values with clean_cells function
    data_clean = data_clean.applymap(lambda x: clean_cells(x))
    # Drop column with variable name like "age,isced11,unit,sex,geo\time". It is the first column. we have a 
    # dataframe with only data values 
    data_clean.drop(data_clean.columns[0], axis = 1, inplace = True)
    # transform column with variable in multiple-columns  
    variabili = data[data.columns[0]].apply(lambda x: pd.Series(x.split(",")))
    variabili.columns = data.columns[0].split(",")
    # return cleaned dataframe in pandas dataframe
    return pd.concat([variabili, data_clean], axis = 1)

In [80]:
def tsv_to_dataframe_long(file_tsv):
    '''
    this function transforms Eurostat tsv file in pandas dataframe
    file_tsv: file name. It's work with tsv and compressed file "tsv.gz"
    '''
    
    def clean_cells(x):
        '''This function transforms Eurostat Missing Values ": " in numpy missing values.
        Then clean Eurostat annotation "b, u, .."'''
        try:
            return float(x)
        except:
            try:
                return float(x.split(" ")[0])
            except:
                return np.nan
            
    def annotation(x):
            '''This function extracts Eurostat annotation "b, u, .."'''
            try:
                return x.split(" ")[1]
            except:
                return np.nan

    def columns_type(x):
            try:
                return int(x.strip())
            except:
                if x == "geo\\time":
                    return "var_" + "geo"
                else:
                    return "var_" + x

    
    # open the Eurostat TSV file 

    data = pd.read_csv(file_tsv, sep="\t")
    # Create a dataframe for values data
    data_clean = data
    # Clean data values with clean_cells function
    data_clean = data_clean.applymap(lambda x: x)
    # Drop column with variable name like "age,isced11,unit,sex,geo\time". It is the first column. we have a 
    # dataframe with only data values 
    data_clean.drop(data_clean.columns[0], axis = 1, inplace = True)
    # transform column with variable in multiple-columns  
    variabili = data[data.columns[0]].apply(lambda x: pd.Series(x.split(",")))
    variabili.columns = data.columns[0].split(",")
    # return cleaned dataframe in pandas dataframe
    result = pd.concat([variabili, data_clean], axis = 1)
    colonne  = list(map(lambda x: columns_type(x), list(result.columns.values)))
    result.columns = colonne
    colonne_var = result.columns.map(lambda x: str(x)[0:3] == "var")
    index = list(compress(result.columns, colonne_var))
    result = result.melt(id_vars = index)
    result["value_raw"] = result["value"]
    result["eurostat_annotation"] = result["value_raw"].apply(lambda x: annotation(x))
    result["value"] = result["value"].apply(clean_cells) 
    
    
    return result

In [13]:
#example 
#if __name__ == "__main__":
import urllib.request
import gzip

eurostat_link = "http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&file=data%2Faact_eaa07.tsv.gz"
urllib.request.urlretrieve(eurostat_link , "file.tsv.gz")

('file.tsv.gz', <http.client.HTTPMessage at 0x7f43e0d49f98>)

In [81]:
result = tsv_to_dataframe_long("file.tsv.gz")
result

,var_itm_newa,var_indic_ag,var_unit,var_geo,variable,value,value_raw,eurostat_annotation
0,01000,PROD_BP,MIO_EUR,AT,2017,857.51,857.51 e,e
1,01000,PROD_BP,MIO_EUR,BE,2017,393.61,393.61 e,e
2,01000,PROD_BP,MIO_EUR,BG,2017,1136.57,1136.57 e,e
3,01000,PROD_BP,MIO_EUR,CH,2017,290.24,290.24 e,e
4,01000,PROD_BP,MIO_EUR,CY,2017,11.99,11.99 e,e
5,01000,PROD_BP,MIO_EUR,CZ,2017,1062.51,1062.51 e,e
6,01000,PROD_BP,MIO_EUR,DE,2017,4530.67,4530.67 e,e
7,01000,PROD_BP,MIO_EUR,DK,2017,1761.17,1761.17 e,e
8,01000,PROD_BP,MIO_EUR,EA11,2017,26275.97,26275.97 e,e
9,01000,PROD_BP,MIO_EUR,EA12,2017,27068.94,27068.94 e,e


In [68]:
def clean_cells(x):
        '''This function transforms Eurostat Missing Values ": " in numpy missing values.
        Then clean Eurostat annotation "b, u, .."'''
        try:
            return float(x)
        except:
            try:
                return float(x.split(" ")[0])
            except:
                return np.nan
            
def annotation(x):
        '''This function extracts Eurostat annotation "b, u, .."'''
        try:
            return x.split(" ")[1]
        except:
            return np.nan

def columns_type(x):
        try:
            return int(x.strip())
        except:
            if x == "geo\\time":
                return "var_" + "geo"
            else:
                return "var_" + x

colonne  = list(map(lambda x: columns_type(x), list(result.columns.values)))



In [69]:
result.columns = colonne

In [70]:
colonne_var = result.columns.map(lambda x: str(x)[0:3] == "var")
from itertools import compress
index = list(compress(result.columns, colonne_var))


In [71]:
index

['var_itm_newa', 'var_indic_ag', 'var_unit', 'var_geo']

In [72]:
result = result.melt(id_vars = index)

In [73]:
result

,var_itm_newa,var_indic_ag,var_unit,var_geo,variable,value
0,01000,PROD_BP,MIO_EUR,AT,2017,857.51 e
1,01000,PROD_BP,MIO_EUR,BE,2017,393.61 e
2,01000,PROD_BP,MIO_EUR,BG,2017,1136.57 e
3,01000,PROD_BP,MIO_EUR,CH,2017,290.24 e
4,01000,PROD_BP,MIO_EUR,CY,2017,11.99 e
5,01000,PROD_BP,MIO_EUR,CZ,2017,1062.51 e
6,01000,PROD_BP,MIO_EUR,DE,2017,4530.67 e
7,01000,PROD_BP,MIO_EUR,DK,2017,1761.17 e
8,01000,PROD_BP,MIO_EUR,EA11,2017,26275.97 e
9,01000,PROD_BP,MIO_EUR,EA12,2017,27068.94 e


In [74]:
# a questo punto si dovrebbe creare due colonne= una per i valori puliti, l'altra per l'annotazione
result["value_raw"] = result["value"]

In [75]:
annotation("1232 :u")

':u'

In [77]:
result["eurostat_annotation"] = result["value_raw"].apply(lambda x: annotation(x))

In [78]:
result["value"] = result["value"].apply(clean_cells) 

In [79]:
result

,var_itm_newa,var_indic_ag,var_unit,var_geo,variable,value,value_raw,eurostat_annotation
0,01000,PROD_BP,MIO_EUR,AT,2017,857.51,857.51 e,e
1,01000,PROD_BP,MIO_EUR,BE,2017,393.61,393.61 e,e
2,01000,PROD_BP,MIO_EUR,BG,2017,1136.57,1136.57 e,e
3,01000,PROD_BP,MIO_EUR,CH,2017,290.24,290.24 e,e
4,01000,PROD_BP,MIO_EUR,CY,2017,11.99,11.99 e,e
5,01000,PROD_BP,MIO_EUR,CZ,2017,1062.51,1062.51 e,e
6,01000,PROD_BP,MIO_EUR,DE,2017,4530.67,4530.67 e,e
7,01000,PROD_BP,MIO_EUR,DK,2017,1761.17,1761.17 e,e
8,01000,PROD_BP,MIO_EUR,EA11,2017,26275.97,26275.97 e,e
9,01000,PROD_BP,MIO_EUR,EA12,2017,27068.94,27068.94 e,e


In [ ]:
print(result.dtypes)

In [5]:
# Lib example

In [6]:
import tsv_to_pandas
import urllib.request
import gzip

#bulkdownload http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing
eurostat_link = "http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/AppLinkServices?lang=en&appId=bulkdownload&appUrl=http%3A%2F%2Fec.europa.eu%2Feurostat%2Festat-navtree-portlet-prod%2FBulkDownloadListing%3Ffile%3Ddata%2Frd_e_gerdtot.tsv.gz"
urllib.request.urlretrieve(eurostat_link , "file.tsv.gz")
tsv_to_pandas.tsv_to_dataframe("file.tsv.gz")

,sectperf,unit,geo\time,2016,2015,2014,2013,2012,2011,2010,...,1990,1989,1988,1987,1986,1985,1984,1983,1982,1981
0,BES,EUR_HAB,AT,896.3,874.3,856.2,802.0,777.9,679.7,661.0,...,NaN,121.6,NaN,NaN,NaN,79.5,73.2,NaN,NaN,51.5
1,BES,EUR_HAB,BA,NaN,NaN,NaN,NaN,5.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BES,EUR_HAB,BE,648.2,629.7,597.1,570.7,555.2,510.3,463.8,...,NaN,159.3,156.5,150.6,140.4,128.9,115.2,105.2,96.0,92.1
3,BES,EUR_HAB,BG,38.4,44.3,30.8,22.4,21.0,15.9,14.6,...,16.8,54.6,48.1,35.3,NaN,NaN,NaN,NaN,NaN,NaN
4,BES,EUR_HAB,CH,NaN,1780.2,NaN,NaN,1489.0,NaN,NaN,...,NaN,521.1,NaN,NaN,477.1,NaN,NaN,288.4,NaN,226.3
5,BES,EUR_HAB,CN_X_HK,NaN,113.5,89.9,81.7,71.5,54.3,43.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BES,EUR_HAB,CY,35.5,23.0,23.3,19.6,16.2,16.4,18.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BES,EUR_HAB,CZ,171.7,167.5,164.5,154.2,146.8,134.6,115.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BES,EUR_HAB,DE,764.5,750.7,705.7,665.2,669.6,636.7,573.7,...,373.2,360.7,341.7,326.3,296.1,266.4,230.6,215.1,195.2,169.0
9,BES,EUR_HAB,DK,918.8,908.0,877.6,868.9,891.8,875.8,859.1,...,183.3,158.4,147.9,136.8,120.4,103.7,90.1,78.2,65.7,54.7
